Installazione delle librerie aggiuntive necessarie

*   `emoji`: fornisce funzionalità per lavorare con emoticon e caratteri emoji in Python
*   `fasttext`: libreria per calcolare gli embedding di parole, utilizzata in uno dei diversi approcci del data augmentation
* `nlpaug`: libreria per effettuare data augmentation di testo. Essa offre tecniche per generare varianti del testo originale
*   `sentence-transformers`: framework per calcolare gli embeddings di frasi

In [ ]:
!pip install emoji
!pip install fasttext
!pip install nlpaug
!pip install sentence-transformers

Import delle librerie utili.

In [ ]:
import pandas as pd
import numpy as np
import emoji
import nltk
import random
import fasttext.util
import gc
import nlpaug.augmenter.word as naw
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from torch import nn
from tqdm.auto import tqdm
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from nltk.corpus import wordnet as wn
from transformers import AutoConfig, AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics import f1_score
from google.colab import drive

In [ ]:
root = '/content/drive/MyDrive/Colab Notebooks/Competizione/data/taskB/'
drive.mount('/content/drive')

Vengono caricati i dati di train.

In [ ]:
train_B = pd.read_csv(root + "subtaskB_train.csv", index_col="Id")
df_train_B = train_B.copy()

**POSSIBILI FEATURES IN PIÙ**

Si definiscono i metodi che servono per valutare le eventuali features aggiuntive per ogni sentence.

In [ ]:
def check_keyword(sentence, key):
    sentence=sentence.split()
    for word in sentence:
        if word.lower() == key:
            return 1
    return 0

def check_stem(sentence, stem):
    sentence=sentence.split()
    for word in sentence:
      word=word.lower()
      if len(word) >=len(stem) and word[:len(stem)]==stem:
        return 1
    return 0

def check_emoji(sentence, e):
    emoji_list = emoji.distinct_emoji_list(sentence)
    if e in emoji_list:
        return 1
    return 0

Viene definito il dizionario delle features aggiuntive.

In [ ]:
features = {
    "covid" :[],
    "sier": [],
    "vaccin": [],
    "green pass": [],
    "sars": [],
    "terra": [],
    "piatta": [],
    "russia": [],
    "ucraina": [],
    "putin": [],
    "kiev": [],
    "mosca": [],
    "trump": [],
    "sole" : []
}

Metodi per istanziare le features nel dizionario e per ottenere i valori delle features in base ad una sentence.

In [ ]:
def empty_features(features):
    for key in features.keys():
        features[key] = []

def get_features(sentence, features):
    features["covid"].append(check_keyword(sentence, "covid"))
    features["sier"].append(check_stem(sentence, "sier"))
    features["vaccin"].append(check_stem(sentence, "vaccin"))
    features["green pass"].append(check_keyword(sentence, "green pass"))
    features["sars"].append(check_stem(sentence, "sars"))
    features["terra"].append(check_keyword(sentence, "terra"))
    features["piatta"].append(check_keyword(sentence, "piatta"))
    features["russia"].append(check_keyword(sentence, "russia"))
    features["ucraina"].append(check_keyword(sentence, "ucraina"))
    features["putin"].append(check_keyword(sentence, "putin"))
    features["kiev"].append(check_keyword(sentence, "kiev"))
    features["mosca"].append(check_keyword(sentence, "mosca"))
    features["trump"].append(check_keyword(sentence, "trump"))
    features["sole"].append(check_keyword(sentence, "sole"))

    for e in list(features.keys())[14:len(features)]:
        features[e].append(check_emoji(sentence, str(e)))

Il seguente frammento di codice genera un dataframe con le emoji più usate, quindi presenti in almeno 10 frasi distinte.

In [ ]:
emojis={}
for sentence in df_train_B.comment_text:
  sentence_emojis=emoji.distinct_emoji_list(sentence)
  for e in sentence_emojis:
    emojis[e]=0

for sentence in df_train_B.comment_text:
  sentence_emojis=emoji.distinct_emoji_list(sentence)
  for e in sentence_emojis:
    emojis[e]+=1

df_emoji=pd.DataFrame(emojis, index=['frequence']).T
df_frequent=df_emoji[df_emoji['frequence']>=10].T

Ogni emoji calcolata viene aggiunta alle features.

In [ ]:
for column in df_frequent.columns:
  features[column]=[]

**SPLIT DATI**

Split dei dati in training set e validation set. In questo caso il validation set viene utilizzato per far sì che l'addestramento non vada in overfitting.

In [ ]:
(x_train, x_val, y_train, y_val) = train_test_split(df_train_B.comment_text, df_train_B.conspiracy, test_size=0.15, random_state=42)

**DATA AUGMENTATION**

Prima di scegliere la libreria nlpaug per effettuare data augmentation sono state fatte altre prove per aumentare i dati.

Primo approccio: utilizzo di WordNet per ottenere la lista di sinonimi, generazione degli embeddings mediante fastText e uso della distanza coseno per ottenere il sinonimo più "vicino".

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

ft = fasttext.load_model(root+'fastText/cc.it.300.bin')

def data_augmentation(testo_originale):
    testo = testo_originale
    parole = nltk.word_tokenize(testo_originale)            #lista di parole che compongono la frase originale
    n_parole = random.randint(1, int(len(parole)/2 + 1))    #numero di parole da modificare
    indici = random.sample(range(len(parole)), n_parole)    #indici delle parole da modificare
    parole_sinonimi = [parole[i] for i in indici]           #parole da modificare
    for parola in parole_sinonimi:
        testo = sostituisci_sinonimo(testo, parola)
    if testo != testo_originale:
        return testo
    else:
        return testo_originale

def sinonimi(parola):
    sinonimi = wn.synonyms(parola, lang='ita')          #tutti i sinonimi della parola
    s = [elem for lista in sinonimi for elem in lista]
    return s

def sostituisci_sinonimo(testo, parola):
    sinonimi_parola = sinonimi(parola)
    for elem in sinonimi_parola:
        if elem.lower() == parola.lower():      #eliminazione dei sinonimi dove cambia solo il modo in cui è scritta la parola
            sinonimi_parola.remove(elem)
    if sinonimi_parola != []:
        elemento_scelto = scelta_elemento(parola, sinonimi_parola)
        testo_modificato = testo.replace(parola, elemento_scelto)
        return testo_modificato
    else:
        return testo    #se non ci sono sinonimi per le parole scelte, viene restituito il testo originale

def scelta_elemento(parola, sinonimi_parola):
    embeddings_originale = ft[parola]       #calcolo embeddings
    distanze = [cosine(embeddings_originale, ft[elem]) for elem in sinonimi_parola]
    sinonimo = sinonimi_parola[np.argmin(distanze)]     #si prende il sinonimo più vicino
    return sinonimo

Esempio di utilizzo

In [ ]:
'''
frase_originale =  "Vado a fare una passeggiata con il cane"
frase_nuova = frase_originale
while frase_nuova == frase_originale:
    frase_nuova = data_augmentation(frase_originale)

print("Frase originale:", frase_originale)
print("Frase aumentata:", frase_nuova)
'''

Secondo approccio: utilizzo di fastText per ottenere il sinonimo più vicino dal punto di vista della rappresentazione vettoriale.

In [ ]:
def data_augmentation(testo_originale):
    testo = testo_originale
    parole = nltk.word_tokenize(testo_originale)            #lista di parole che compongono la frase originale
    n_parole = random.randint(1, int(len(parole)/2 + 1))    #numero di parole da modificare
    indici = random.sample(range(len(parole)), n_parole)    #indici delle parole da modificare
    parole_sinonimi = [parole[i] for i in indici]           #parole da modificare
    for parola in parole_sinonimi:
        testo = sostituisci_sinonimo(testo, parola)
    if testo != testo_originale:                            #se il testo originale non ha subito cambiamenti, esso viene restituito, così da poter effettuare una verifica prima di considerare la frase restituita come nuova
        return testo
    else:
        return testo_originale

def sostituisci_sinonimo(testo, parola):
    elemento_scelto = scelta_elemento(parola)
    if elemento_scelto.lower() != parola.lower():           #per non scegliere come sinonimo la stessa parola scritta diversamente
        testo_modificato = testo.replace(parola, elemento_scelto)
        return testo_modificato
    else:
        return testo

def scelta_elemento(parola):
    return ft.get_nearest_neighbors(parola)[0][1]           #ritorna il sinonimo più vicino

Esempio di utilizzo

In [ ]:
'''
frase_originale =  "Vado a fare una passeggiata con il cane"
frase_nuova = frase_originale
while frase_nuova == frase_originale:
    frase_nuova = data_augmentation(frase_originale)

print("Frase originale:", frase_originale)
print("Frase aumentata:", frase_nuova)
'''

Dopo il caricamento del modello di fastText, la RAM risulta quasi satura, quindi prima di continuare l'esecuzione del codice, poichè ft non servirà più, si può procedere alla liberazione della RAM stessa.

In [ ]:
ft = None
gc.collect()

<u>Approccio utilizzato per il data augmentation</u>: libreria nlpaug

Viene effettuato il data augmentation andando ad utilizzare embeddings contestuali (BERT).

In particolare il data augmentation considerato in questo caso inserisce delle nuove parole nel testo.

In [ ]:
aug = naw.ContextualWordEmbsAug(model_path="dbmdz/bert-base-italian-xxl-uncased", action='insert', device='cuda')
augmented = aug.augment(list(x_train))

Vengono create le liste con i testi aumentati e le labels associate.

In [ ]:
df_augmented = list(x_train) + augmented
df_labels = list(y_train) + list(y_train)

**IPERPARAMETRI**

Viene definito il dizionario degli iperparametri, in modo tale da dover modificare solo i valori per effettuare addestramenti diversi.

In [ ]:
n_labels = 4

model_name = "dbmdz/bert-base-italian-xxl-uncased"

hyperparameters = {
    "epochs": 1000,
    "learning_rate": 1e-6,
    "batch_size": 16,
    "dropout": 0.1,
    "h_dim": 768,
    "patience": 50,
    "min_delta": 0.001,
    "language_model": model_name,
    "extra-features" : True
}

if hyperparameters["extra-features"] == False:
    features = {}

**DATASET**

La classe Dataset permette di ottenere un oggetto che contenga sia i testi sia le labels.

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, x, y):
        self.texts = [text for text in x]
        self.labels = [torch.tensor(label) for label in y]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        batch_texts = self.texts[idx]
        batch_labels = np.array(self.labels[idx])
        return batch_texts, batch_labels

**CLASSIFICATORE**

Viene definita la rete neurale che ha il compito di effettuare la classificazione.


È composta da:
*   Layer lineare con input e output di dimensione uguale a quella del vettore in ingresso
*   Layer di normalizzazione della batch
*   Layer di dropout
*   Layer con funzione di attivazione ReLU
*   Layer lineare con input uguale alla dimensione del vettore in ingresso e output uguale a 4, ossia il numero di labels

In [ ]:
class ClassifierDeep(nn.Module):

    def __init__(self, labels, hdim, dropout):
        super(ClassifierDeep, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(hdim, hdim),
            nn.BatchNorm1d(hdim),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(hdim, labels)
        )

    def forward(self, input_texts):
        return self.classifier(input_texts)

**EARLY STOPPING**

Definita la classe EarlyStopping, utilizzata come funzione di callback che si occupa di interrompere l'addestramento nel caso in cui la loss dovesse avere un trend convergente in cui il valore rimane nell'intervallo del min_delta, oppure in salita. Inoltre salva progressivamente il modello migliore.

In [ ]:
class EarlyStopping:
    def __init__(self, patience, min_delta):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False
        self.min_loss = torch.inf

    def __call__(self, loss, model, path):
        if self.min_loss > (loss + self.min_delta):
            self.min_loss = loss
            self.counter = 0
            torch.save(model, path)
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                print("Early stopping.")

**ADDESTRAMENTO**

Per quanto riguarda l'addestramento abbiamo utilizzato 2 approcci: embeddings calcolati sulle singole parole ed embeddings calcolati sull'intera sentence.

Primo approccio: embeddings calcolati sulle singole parole.

Metodo per la generazione degli embeddings calcolati sulle singole parole. Il metodo restituisce l’output del last_hidden_state del modello.

In [ ]:
def gen_embeddings(input_id_text, attention_mask, lm_model):
    with torch.no_grad():
        last_hidden_states = lm_model(input_id_text, attention_mask=attention_mask).last_hidden_state
        last_hidden_states = last_hidden_states[:,0,:]
    return last_hidden_states

Train loop.

Per ogni sentence nella batch vengono generati i token dal tokenizer e attraverso il language model vengono generati gli embeddings dei token.
Si concatenano ai vettori degli embeddings le eventuali features aggiuntive.

In [ ]:
def train_loop(model, dataloader, lm_model, tokenizer, loss, optimizer, device, features):
    model.train()

    epoch_acc = 0
    epoch_loss = 0
    epoch_labels, epoch_output = [], []

    for batch_texts, batch_labels in tqdm(dataloader, desc='training set', position=0, leave=True):

        optimizer.zero_grad()
        batch_labels = batch_labels.to(device)
        tokens = tokenizer(list(batch_texts), add_special_tokens=True, return_tensors='pt', padding='max_length', max_length = 512, truncation=True)
        input_id_texts = tokens['input_ids'].squeeze(1).to(device)
        mask_texts = tokens['attention_mask'].squeeze(1).to(device)
        embeddings_texts = gen_embeddings(input_id_texts, mask_texts, lm_model)

        if features != {}:
            empty_features(features)
            for sentence in list(batch_texts):
                get_features(sentence, features)
            df_features=pd.DataFrame(features)
            for column in df_features.columns:
                new_column = torch.tensor(df_features[column].values, dtype=torch.float32).view(-1, 1).to(device)
                embeddings_texts = torch.cat((embeddings_texts, new_column), dim=1)

        embeddings_texts = embeddings_texts.clone().to(torch.float32).to(device)
        batch_output = model(embeddings_texts)

        batch_loss = loss(batch_output, batch_labels)
        epoch_loss += batch_loss.item()
        batch_loss.backward()
        optimizer.step()

        softmax = nn.LogSoftmax(dim=1)
        batch_output = softmax(batch_output).argmax(dim=1)
        epoch_acc += (batch_output == batch_labels).sum().item()

        batch_labels = batch_labels.detach().cpu()
        batch_output = batch_output.detach().cpu()
        input_id_texts = input_id_texts.detach().cpu()
        mask_texts = mask_texts.detach().cpu()

        epoch_labels += batch_labels

        batch_output = list(batch_output.numpy().astype(int))
        epoch_output += batch_output

    return epoch_loss/len(dataloader), epoch_acc, epoch_labels, epoch_output

Validation loop.

Stesse operazioni del train loop senza effettuare la backpropagation.

In [ ]:
def val_loop(model, dataloader, lm_model, tokenizer, loss, device, features):
    model.eval()

    epoch_acc = 0
    epoch_loss = 0
    epoch_labels, epoch_output = [], []

    with torch.no_grad():

        for batch_texts, batch_labels, in tqdm(dataloader, desc='validation set', position=0, leave=True):

            batch_labels = batch_labels.to(device)
            tokens = tokenizer(list(batch_texts), add_special_tokens=True, return_tensors='pt', padding='max_length', max_length = 512, truncation=True)
            input_id_texts = tokens['input_ids'].squeeze(1).to(device)
            mask_texts = tokens['attention_mask'].squeeze(1).to(device)
            batch_labels = batch_labels.to(device)
            embeddings_texts = gen_embeddings(input_id_texts, mask_texts, lm_model)

            if features != {}:
                empty_features(features)
                for sentence in list(batch_texts):
                    get_features(sentence, features)
                df_features = pd.DataFrame(features)
                for column in df_features.columns:
                    new_column = torch.tensor(df_features[column].values, dtype=torch.float32).view(-1, 1).to(device)
                    embeddings_texts = torch.cat((embeddings_texts, new_column), dim=1)

            embeddings_texts = embeddings_texts.clone().to(torch.float32).to(device)
            batch_output = model(embeddings_texts)

            batch_loss = loss(batch_output, batch_labels)
            epoch_loss += batch_loss.item()

            softmax = nn.LogSoftmax(dim=1)
            batch_output = softmax(batch_output).argmax(dim=1)
            epoch_acc += (batch_output == batch_labels).sum().item()

            batch_labels = batch_labels.detach().cpu()
            batch_output = batch_output.detach().cpu()
            input_id_texts = input_id_texts.detach().cpu()
            mask_texts = mask_texts.detach().cpu()

            epoch_labels += batch_labels

            batch_output = list(batch_output.numpy().astype(int))
            epoch_output += batch_output

    return epoch_loss/len(dataloader), epoch_acc, epoch_labels, epoch_output

Il metodo per l'addestramento.

Nel metodo vengono utilizzati AutoConfig, AutoTokenizer e AutoModel per configurare il language model. Quest’ultimo viene poi passato come argomento, insieme al tokenizer, ai metodi train_loop e val_loop.

In [ ]:
def train_test(model, epochs, optimizer, device, train_data, val_data, batch_size, model_name, train_loss_fn, early_stopping, features):

    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

    train_loss, validation_loss = [], []
    train_acc, validation_acc = [], []
    train_f1, validation_f1 = [], []

    config = AutoConfig.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    lm_model = AutoModel.from_pretrained(model_name, config=config).to(device)

    for epoch in tqdm(range(1, epochs+1), desc='epoch'):

        epoch_train_loss, epoch_train_acc, train_labels, train_outputs = train_loop(model, train_dataloader, lm_model, tokenizer, train_loss_fn, optimizer, device, features)
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_acc/len(train_data))

        epoch_validate_loss, epoch_validate_acc, val_labels, val_outputs = val_loop(model, val_dataloader, lm_model, tokenizer, train_loss_fn, device, features)
        validation_loss.append(epoch_validate_loss)
        validation_acc.append(epoch_validate_acc/len(val_data))

        epoch_train_f1 = f1_score(train_labels, train_outputs, average='macro')
        train_f1.append(epoch_train_f1)

        epoch_validate_f1 = f1_score(val_labels, val_outputs, average='macro')
        validation_f1.append(epoch_validate_f1)

        print(f"\nTrain loss: {epoch_train_loss:6.4f} Validation loss: {epoch_validate_loss:6.4f}")
        print(f"Train accuracy: {(epoch_train_acc/len(train_data)):6.4f} Validation accuracy: {(epoch_validate_acc/len(val_data)):6.4f}\n")
        print(f"Train f1 score: {epoch_train_f1:6.4f} Validation f1 score: {epoch_validate_f1:6.4f}\n")

        path = root + 'model/best_model.pth'
        early_stopping(epoch_validate_loss, model, path)
        if early_stopping.early_stop:
            break

    return train_loss, validation_loss, train_acc, validation_acc, train_f1, validation_f1

<u>Approccio utilizzato</u>: Sentence Transformers

Metodo per la generazione degli embeddings.

In [ ]:
def gen_embeddings(batch_texts, features, lm_model):
    input = lm_model.encode(batch_texts)
    if features != {}:
        empty_features(features)
        for sentence in list(batch_texts):
            get_features(sentence, features)
        df_features=pd.DataFrame(features)
        for column in df_features.columns:
            input=np.append(input, np.array([list(df_features[column])]).T, axis=1)
    embeddings_texts = torch.tensor(input).to(torch.float32)

    return embeddings_texts

Viene definito il train loop dove vengono effettuate le seguenti operazioni:

1.   calcolati gli embeddings delle sentences tramite il language model (metodo gen_embeddings)
2.   calcolate e concatenate le eventuali features aggiuntive (metodo gen_embeddings)
3.   passato il vettore risultante come input al modello
4.   calcolata la loss con l'output ottenuto

In [ ]:
def train_loop(model, dataloader, lm_model, loss, optimizer, device, features):
    model.train()

    epoch_acc = 0
    epoch_loss = 0

    epoch_labels, epoch_output = [], []

    for batch_texts, batch_labels in tqdm(dataloader, desc='train', position=0, leave=True):

        optimizer.zero_grad()
        batch_labels = batch_labels.to(device)
        embeddings_texts = gen_embeddings(batch_texts, features, lm_model).to(device)
        output = model(embeddings_texts)

        batch_loss = loss(output, batch_labels)
        batch_loss.backward()
        optimizer.step()

        epoch_loss += batch_loss.item()

        softmax = nn.LogSoftmax(dim=1)
        epoch_acc += (softmax(output).argmax(dim=1) == batch_labels).sum().item()

        batch_labels = batch_labels.detach().cpu()
        output = output.detach().cpu()

        epoch_labels += batch_labels

        batch_output = list(softmax(output).argmax(dim=1).numpy().astype(int))
        epoch_output += batch_output

    return epoch_loss/len(dataloader), epoch_acc, epoch_labels, epoch_output

Viene definito il validation loop, in cui vengono eseguite le stesse operazioni del train loop senza effettuare la backpropagation, ma utilizzando il modello solo per la valutazione.

In [ ]:
def val_loop(model, dataloader, lm_model, loss, device, features):
    model.eval()

    epoch_acc = 0
    epoch_loss = 0

    epoch_labels, epoch_output = [], []

    with torch.no_grad():

        for batch_texts, batch_labels, in tqdm(dataloader, desc="val"):
            batch_labels = batch_labels.to(device)
            embeddings_texts = gen_embeddings(batch_texts, features, lm_model).to(device)
            output = model(embeddings_texts)

            batch_loss = loss(output, batch_labels)
            epoch_loss += batch_loss.item()

            softmax = nn.LogSoftmax(dim=1)
            epoch_acc += (softmax(output).argmax(dim=1) == batch_labels).sum().item()

            batch_labels = batch_labels.detach().cpu()
            output = output.detach().cpu()

            epoch_labels += batch_labels

            batch_output = list(softmax(output).argmax(dim=1).numpy().astype(int))
            epoch_output += batch_output

    return epoch_loss/len(dataloader), epoch_acc, epoch_labels, epoch_output

Il metodo seguente effettua i loop di train e validation e ritorna la loss, l'accuratezza e la f1-score per epoca.

In [ ]:
def train_test(model, epochs, optimizer, device, train_data, val_data, batch_size, model_name, train_loss_fn, early_stopping, features):

    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)

    train_loss, validation_loss = [], []
    train_acc, validation_acc = [], []
    train_f1, validation_f1 = [], []

    lm_model = SentenceTransformer(hyperparameters['language_model'])

    for epoch in tqdm(range(1, epochs+1), desc='epoch'):

        epoch_train_loss, epoch_train_acc, train_labels, train_outputs = train_loop(model, train_dataloader, lm_model, train_loss_fn, optimizer, device, features)
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_acc/len(train_data))

        epoch_validate_loss, epoch_validate_acc, val_labels, val_outputs = val_loop(model, val_dataloader, lm_model, train_loss_fn, device, features)
        validation_loss.append(epoch_validate_loss)
        validation_acc.append(epoch_validate_acc/len(val_data))

        epoch_train_f1 = f1_score(train_labels, train_outputs, average='macro')
        train_f1.append(epoch_train_f1)

        epoch_validate_f1 = f1_score(val_labels, val_outputs, average='macro')
        validation_f1.append(epoch_validate_f1)

        print(f"\nTrain loss: {epoch_train_loss:6.4f} Validation loss: {epoch_validate_loss:6.4f}")
        print(f"Train accuracy: {(epoch_train_acc/len(train_data)):6.4f} Validation accuracy: {(epoch_validate_acc/len(val_data)):6.4f}\n")
        print(f"Train f1 score: {epoch_train_f1:6.4f} Validation f1 score: {epoch_validate_f1:6.4f}\n")

        path = root + 'model/best_model.pth'
        early_stopping(epoch_validate_loss, model, path)
        if early_stopping.early_stop:
            break

    return train_loss, validation_loss, train_acc, validation_acc, train_f1, validation_f1

Vengono creati gli oggetti Dataset per ogni set di dati.

In [ ]:
train_dataset = Dataset(df_augmented, df_labels)
val_dataset = Dataset(x_val, y_val)

Vengono istanziate le seguenti variabili per cominciare l'addestramento:

*   il device utilizzato (per l'addestramento è stata usata la GPU)
*   il modello della rete classificatore
*   la funzione di loss da utilizzare, la Cross Entropy, in quanto siamo dinanzi ad una classificazione multiclasse
*   l'ottimizzatore, Adam
*   la funzione di callback, l'early-stopping

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassifierDeep(n_labels, hyperparameters["h_dim"]+len(features)*hyperparameters["extra-features"], hyperparameters["dropout"]).to(device)
loss = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=hyperparameters["learning_rate"])
early_stopping = EarlyStopping(patience=hyperparameters['patience'], min_delta=hyperparameters['min_delta'])

Viene effettuato l'addestramento.

In [ ]:
train_loss, validation_loss, train_acc, validation_acc, train_f1, validation_f1 = train_test(model, hyperparameters['epochs'], optimizer, device, train_dataset, val_dataset, hyperparameters['batch_size'], hyperparameters['language_model'], loss, early_stopping, features)

**VISUALIZZAZIONE DEI RISULTATI**

Andamento dell'addestramento dal punto di vista numerico.

In [ ]:
print("Train loss: ", train_loss)
print("Validation loss: ", validation_loss)
print("Train accuracy: ", train_acc)
print("Validation accuracy: ", validation_acc)
print("F1 score training set: ", train_f1)
print("F1 score validation set: ", validation_f1)

Vengono plottate le misure ritornate dall'addestramento, sia per il training set sia per il validation set.

In [ ]:
sns.lineplot(data=train_loss, label="train_loss", color="blue")
sns.lineplot(data=validation_loss, label="val_loss", color="red")
plt.legend(loc='upper left')
plt.ylim(0,1)
plt.yticks([i/10 for i in range(11)])
plt.show()


sns.lineplot(data=train_acc, label="train_accuracy", color="blue")
sns.lineplot(data=validation_acc, label="val_accuracy", color="red")
plt.legend(loc='lower right')
plt.ylim(0,1)
plt.yticks([i/10 for i in range(11)])
plt.show()

sns.lineplot(data=train_f1, label="f1-score train", color="blue")
sns.lineplot(data=validation_f1, label="f1-score validation", color="red")
plt.legend(loc='lower right')
plt.ylim(0,1)
plt.yticks([i/10 for i in range(11)])
plt.show()

**PREDIZIONE DELLE ETICHETTE DEL TEST**

Viene caricato il test set.

In [ ]:
df_test_B = pd.read_csv(root + "subtaskB_test.csv", index_col="Id")

Viene caricato il modello salvato.

In [ ]:
path = root + 'model/best_model.pth'
model = torch.load(path)
model.eval()

Vengono predette le etichette dei dati di test.

In [ ]:
lm_model = SentenceTransformer(hyperparameters['language_model'])
input = lm_model.encode(df_test_B.comment_text)
if features != {}:
    empty_features(features)
    for sentence in df_test_B.comment_text:
        get_features(sentence, features)
    df_features=pd.DataFrame(features)
    for column in df_features.columns:
        input=np.append(input, np.array([list(df_features[column])]).T, axis=1)
embeddings_texts = torch.tensor(input).to(torch.float32).to('cuda')
output = model(embeddings_texts)
softmax = nn.LogSoftmax(dim=1)
y_pred = softmax(output).argmax(dim=1).int().tolist()

Vengono salvati i dati di test etichettati dal modello in un file csv per la submission.

In [ ]:
df_new = pd.DataFrame(y_pred, columns=['Expected'])
df_new.to_csv(root + 'test/test.csv', index=True, index_label="Id")